## CRUD con Embeddings

Instalar dependencias en caso de no haber ejecutado el archivo requirements.txt

In [ ]:
%pip install sentence-transformers numpy pandas qdrant-client

Importar librerías necesarias

In [96]:
import random
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance


Crear el modelo de embeddings

In [3]:
modelo = SentenceTransformer('all-MiniLM-L6-v2')


Inicializamos el cliente de Qdrant

In [50]:
client = QdrantClient(
    url="https://c0ecc735-4daa-4faf-8e47-87ec35ba28bc.europe-west3-0.gcp.cloud.qdrant.io:6333", 
    https=True,
    api_key="zXQmNpc6rGfyeSiibzntmDDph_xegr3cI1DoiK5hvK6Uj2WYGKcTdA",
)
index_name = "embeddings_index"

1. Crear un embedding y agregarlo a Qdrant

In [97]:
def generar_id_numerico():
    return random.randint(1000, 9999)


def agregar_embedding(texto):
    # Generar el embedding
    embedding = modelo.encode([texto])[0]
    
    # Generar un ID aleatorio
    nuevo_id = generar_id_numerico()

    client.upsert(
        collection_name=index_name,
        points=[{
            'id': nuevo_id,
            'vector': embedding.tolist(),
            'payload': {'texto': texto}
        }]
    )
    return nuevo_id, embedding

2. Leer un embedding (consultar todos los embeddings)

In [101]:
def obtener_embeddings():
    embeddings = []
    scroll_response = client.scroll(collection_name=index_name, limit=1000)

    while scroll_response and scroll_response[0]:
        for point in scroll_response[0]:
            embeddings.append({
                "id": point.id, 
                "texto": point.payload["texto"]
            })

        # Continuamos con la siguiente página utilizando el scroll_token
        scroll_token = scroll_response[1]
        if not scroll_token:
            break

        # Continuamos el scroll utilizando el token de paginación
        scroll_response = client.scroll(collection_name=index_name, scroll_token=scroll_token)

    return embeddings

3. Consultar los embeddings más similares a una consulta

In [28]:
def consultar_similares(texto_consulta, top_k=5):
    embedding_consulta = modelo.encode([texto_consulta])[0]
    resultados = client.search(
        collection_name=index_name,
        query_vector=embedding_consulta.tolist(),
        limit=top_k
    )
    
    # Extraer los resultados y formatearlos
    return [{
        "id": res.id,
        "texto": res.payload["texto"],
        "score": res.score
    } for res in resultados]

4. Actualizar un embedding por ID

In [29]:
def actualizar_embedding(id, nuevo_texto):
    embedding_nuevo = modelo.encode([nuevo_texto])[0]
    
    client.upsert(
        collection_name=index_name,
        points=[{
            'id': id,
            'vector': embedding_nuevo.tolist(),
            'payload': {'texto': nuevo_texto}
        }]
    )
    return id, nuevo_texto

5. Eliminar un embedding por ID

In [85]:
def eliminar_embedding(id):
    client.delete(
        collection_name=index_name,
        points_selector=[id]
    )
    return id



# Uso del CRUD

1. Crear embeddings

In [98]:
id1, embedding1 = agregar_embedding("Este es un texto de prueba.")
id2, embedding2 = agregar_embedding("Otro texto para generar un embedding.")
id3, embedding3 = agregar_embedding("Ejemplo de texto para el crud.")

2. Leer los embeddings

In [ ]:
print("Embeddings en la base de datos:")
print(obtener_embeddings())

3. Consultar los embeddings más similares a un nuevo texto

In [ ]:
resultados = consultar_similares("texto de ejemplo")
print("\nEmbeddings más similares a la consulta:")
for resultado in resultados:
    print(resultado)

4. Actualizar un embedding

In [ ]:
id_actualizar = 2
nuevo_texto = "Un texto generado para comprobar el funcionamiento del embedding."
id_actualizado, texto_actualizado = actualizar_embedding(id_actualizar, nuevo_texto)

if id_actualizado is not None:
    print(f"Embedding actualizado: {id_actualizado}, Nuevo texto: {texto_actualizado}")
else:
    print("Hubo un error al actualizar el embedding.")

5. Eliminar un embedding

In [ ]:
id = 9167 #reemplazar por el id a eliminar
id_eliminado = eliminar_embedding(id)
print(f"\nEmbedding eliminado: {id_eliminado}")